# Example Usage of Marlowe Oracle

This example uses the Marlowe oracle to read a random value into a contract and pay that amount of ada to a recipient.

The oracle resides at `addr_test1qzy4aluz9zcmxuhuqgjhl457ee0ma0hcq38w62ckaagjwgvnjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wq7a2gly` on the `preprod` network. See https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-apps/Oracle.md for instructions on using the oracle.

![Oracle Example](example-oracle-contract.png)

## Setup

### Select the preproduction network

In [1]:
export CARDANO_NODE_SOCKET_PATH=node.socket
export CARDANO_TESTNET_MAGIC=1

### Set the location of Marlowe Runtime

In [2]:
export MARLOWE_RT_HOST=127.0.0.1
export MARLOWE_RT_PORT=13700

### Set the location of Marlowe executables

In [3]:
MARLOWE_RT_CLI=marlowe-runtime-cli
MARLOWE_CLI=marlowe-cli

### Constants

In [4]:
ADA=1000000

## Addresses

### Oracle address

In [5]:
ORACLE_ADDR=addr_test1qzy4aluz9zcmxuhuqgjhl457ee0ma0hcq38w62ckaagjwgvnjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wq7a2gly

### Address for "c.marlowe" role

In [6]:
PAYMENT_ADDR=$(cat payment.testnet.address)
PAYMENT_SKEY=payment.skey
echo "PAYMENT_ADDR = $PAYMENT_ADDR"

PAYMENT_ADDR = addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


In [7]:
ROLE_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
ROLE_NAME=c.marlowe

### Address for recipient of funds

In [8]:
RECIPIENT_ADDR=addr_test1qq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60slacjyp

## Contract

In [9]:
yaml2json << EOI > example-oracle-contract.json
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
when:
- case:
    deposits:
      multiply: 100
      times: 1000000
    into_account:
      role_token: $ROLE_NAME
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: c.marlowe
  then:
    timeout: $((1000 * (`date -u +%s` + 2 * 60 * 60)))
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 50
          to: 100
        for_choice:
          choice_name: RANDOM
          choice_owner:
            address: "$ORACLE_ADDR"
      then:
        from_account:
          role_token: c.marlowe
        pay:
          multiply:
            value_of_choice:
              choice_name: RANDOM
              choice_owner:
                address: "$ORACLE_ADDR"
          times: 1000000
        then: close
        to:
          party:
            address: "$RECIPIENT_ADDR"
        token:
          currency_symbol: ''
          token_name: ''
EOI
cat example-oracle-contract.json

{"timeout":1689788076000,"timeout_continuation":"close","when":[{"case":{"deposits":{"multiply":100,"times":1000000},"into_account":{"role_token":"c.marlowe"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"c.marlowe"}},"then":{"timeout":1689791676000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":50,"to":100}],"for_choice":{"choice_name":"RANDOM","choice_owner":{"address":"addr_test1qzy4aluz9zcmxuhuqgjhl457ee0ma0hcq38w62ckaagjwgvnjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wq7a2gly"}}},"then":{"from_account":{"role_token":"c.marlowe"},"pay":{"multiply":{"value_of_choice":{"choice_name":"RANDOM","choice_owner":{"address":"addr_test1qzy4aluz9zcmxuhuqgjhl457ee0ma0hcq38w62ckaagjwgvnjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wq7a2gly"}}},"times":1000000},"then":"close","to":{"party":{"address":"addr_test1qq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60slacjyp"}},"token":{"currency_symbol":"","token_

## Transactions

### 1. Create the contract

In [10]:
CONTRACT_ID=$(
"$MARLOWE_RT_CLI" create \
  --core-file example-oracle-contract.json \
  --role-token-policy-id "$ROLE_POLICY" \
  --min-utxo "$((2 * ADA))" \
  --change-address "$PAYMENT_ADDR" \
  --manual-sign tx-1.unsigned \
  2> /dev/null \
| jq -r 'fromjson | .contractId' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 8c1f3c01d8e7c388b61b387395a7890d1299b60ee7bac6f1d066cfb17a475ac6#1


In [11]:
"$MARLOWE_CLI" transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$PAYMENT_SKEY" \
  --timeout 600s

TxId "8c1f3c01d8e7c388b61b387395a7890d1299b60ee7bac6f1d066cfb17a475ac6"


### 2. Deposit the 100 ada

In [12]:
"$MARLOWE_RT_CLI" deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$ROLE_NAME" \
  --to-party "$ROLE_NAME" \
  --lovelace "$((100 * ADA))" \
  --change-address "$PAYMENT_ADDR" \
  --manual-sign tx-2.unsigned \
| jq 'fromjson' | json2yaml

txId: 192a0db14566e8c81c65c2342fe2e1571dce74669810e9895d34a864777ac2c9


In [13]:
$MARLOWE_CLI transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$PAYMENT_SKEY" \
  --timeout 600s

TxId "192a0db14566e8c81c65c2342fe2e1571dce74669810e9895d34a864777ac2c9"


### Wait for the oracle to report a random value to the contract

In [14]:
sleep 3m

### 3. View the report by the oracle

In [15]:
"$MARLOWE_RT_CLI" log \
  --show-contract \
  "$CONTRACT_ID"

transaction 8c1f3c01d8e7c388b61b387395a7890d1299b60ee7bac6f1d066cfb17a475ac6 (creation)
ContractId:      8c1f3c01d8e7c388b61b387395a7890d1299b60ee7bac6f1d066cfb17a475ac6#1
SlotNo:          34101303
BlockNo:         1178418
BlockId:         6a99f47af1ffa8d2408fa574b80b2580063b88121e46c25371edf1cc9717f8b9
ScriptAddress:   addr_test1wqhdyccahvnheppng3fut3phhp3jt5m37zp4529ezz535ms2u9jqv
Marlowe Version: 1

    When [
      (Case
         (Deposit (Role "c.marlowe") (Role "c.marlowe")
            (Token "" "")
            (MulValue
               (Constant 100)
               (Constant 1000000)))
         (When [
            (Case
               (Choice
                  (ChoiceId "RANDOM" (Address "addr_test1qzy4aluz9zcmxuhuqgjhl457ee0ma0hcq38w62ckaagjwgvnjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wq7a2gly")) [
                  (Bound 50 100)])
               (Pay (Role "c.marlowe")
                  (Party (Address "addr_test1qq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0pu

We see that the oracle has reported the value 91.